# Phase 0: Data Quality & Cleaning Report
## Comprehensive Data Audit and Cohort Flow

**Purpose:** Track all data cleaning decisions, removals, and transformations from raw data to analysis sample.  
**Report Date:** January 19, 2026

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from great_tables import GT

# Configure for detailed output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("="*80)
print("PHASE 0: DATA QUALITY & CLEANING AUDIT")
print("="*80)

## Step 1: Load Raw Data & Initial Assessment

In [ ]:
# Load raw data
df_raw = pd.read_csv("USACHICTSI7435_DATA_LABELS_2025-12-02_1515.csv")

# Initialize tracking
cohort_flow = []

# Record initial state
n_initial = len(df_raw)
n_vars_initial = len(df_raw.columns)

cohort_flow.append({
    'Stage': '1. Raw data loaded',
    'N': n_initial,
,
,
print(f"-" * 60)
print(f"Participants: {n_initial}")
print(f"Variables: {n_vars_initial}")
print(f"\nFirst few column names (showing data quality issues):")
print(df_raw.columns[:10].tolist())

## Step 2: Column Name Cleaning & Standardization

In [ ]:
# Work with a copy
df = df_raw.copy()

# Before cleaning - sample column names
print("\n📋 COLUMN NAME CLEANING")
print("-" * 60)
print(f"\nBefore: Sample of original column names:")
print(df.columns[:5].tolist())

# Clean column names
df.columns = df.columns.astype(str)
df.columns = df.columns.str.replace("'", "")  # Remove quotes
df.columns = df.columns.str.replace('\xa0', ' ')  # Remove non-breaking spaces
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
df.columns = df.columns.str.lower()  # Lowercase for consistency

print(f"\nAfter: Cleaned column names:")
print(df.columns[:5].tolist())

# Rename key variables for easier access
rename_dict = {
    "survey date": "survey_date",
    "what is your age?": "age",
    "what is your biological sex?": "sex",
    "what is your household income?": "income",
    "in what year did you or your family arrive to the united states?": "year_arrived_us"
}

df = df.rename(columns=rename_dict)

print(f"\nVariables renamed for convenience: {list(rename_dict.keys())}")
print(f"No rows excluded in this step (data transformation only)")

## Step 3: Variable Selection & Initial Filtering

In [ ]:
# Define variables needed for analysis
core_cols = [
    "survey_date", 
    "age", 
    "sex", 
    "income", 
    "year_arrived_us"
]

cv_cols_original = [
    "have you been diagnosed with any of the following  (choice=heart failure)",
    "have you been diagnosed with any of the following  (choice=hypertension)",
    "heart attack (choice=yes)",
    "stroke (choice=yes)"
]

# Check which variables exist
all_needed_cols = [col for col in (core_cols + cv_cols_original) if col in df.columns]
missing_vars = set(core_cols + cv_cols_original) - set(all_needed_cols)

print("\n🔍 VARIABLE SELECTION")
print("-" * 60)
print(f"Total variables in raw data: {len(df.columns)}")
print(f"Variables needed: {len(core_cols + cv_cols_original)}")
print(f"Variables available: {len(all_needed_cols)}")
print(f"Variables missing: {len(missing_vars)}")

if missing_vars:
    print(f"\n⚠️  Missing variables: {missing_vars}")

# Select only needed columns
df = df[all_needed_cols].copy()

print(f"\nVariables retained for analysis: {len(df.columns)}")
print(f"No rows excluded (variable selection only)")

## Step 4: Data Type Conversion & Initial Validation

In [ ]:
print("\n🔄 DATA TYPE CONVERSION & VALIDATION")
print("-" * 60)

# Track conversions and issues
conversion_log = []

# 1. Survey Date
print("\n1. SURVEY_DATE")
print(f"   Original type: {df['survey_date'].dtype}")
print(f"   Sample values: {df['survey_date'].head(3).tolist()}")
df["survey_date"] = pd.to_datetime(df["survey_date"], errors="coerce")
invalid_dates = df['survey_date'].isna().sum()
print(f"   After conversion: {df['survey_date'].dtype}")
print(f"   ⚠️  Invalid dates converted to NaN: {invalid_dates}")
if invalid_dates > 0:
    conversion_log.append(f"Survey date: {invalid_dates} invalid values")

# 2. Age
print("\n2. AGE")
print(f"   Original type: {df['age'].dtype}")
print(f"   Sample values: {df['age'].head(3).tolist()}")
df["age"] = pd.to_numeric(df["age"], errors="coerce")
invalid_ages = df['age'].isna().sum()
print(f"   After conversion: {df['age'].dtype}")
print(f"   ⚠️  Non-numeric values converted to NaN: {invalid_ages}")
if df['age'].notna().any():
    print(f"   Range: {df['age'].min():.0f} - {df['age'].max():.0f}")

# 3. Year Arrived US
print("\n3. YEAR_ARRIVED_US")
print(f"   Original type: {df['year_arrived_us'].dtype}")
print(f"   Sample values: {df['year_arrived_us'].head(3).tolist()}")
df["year_arrived_us"] = pd.to_numeric(df["year_arrived_us"], errors="coerce")
invalid_years = df['year_arrived_us'].isna().sum()
print(f"   After conversion: {df['year_arrived_us'].dtype}")
print(f"   ⚠️  Non-numeric values converted to NaN: {invalid_years}")
if df['year_arrived_us'].notna().any():
    print(f"   Range: {df['year_arrived_us'].min():.0f} - {df['year_arrived_us'].max():.0f}")

## Step 5: Outlier Detection & Removal

In [ ]:
print("\n⚡ OUTLIER DETECTION & REMOVAL")
print("-" * 60)

# Track removals
n_before_outliers = len(df)
outlier_exclusions = []

# 1. Age outliers
print("\n1. AGE OUTLIERS (expected range: 0-120)")
age_invalid = ((df["age"] < 0) | (df["age"] > 120) | df["age"].isna())
n_age_invalid = age_invalid.sum()
print(f"   Records with age < 0 or > 120: {n_age_invalid}")
outlier_exclusions.append(('Age out of range', n_age_invalid))

# Keep only valid ages for now (we'll use this for filtering)
df_valid_age = df[~age_invalid].copy()
print(f"   Remaining records: {len(df_valid_age)}")

# 2. Year arrived outliers  
print("\n2. YEAR_ARRIVED_US OUTLIERS (expected range: 1900-2023)")
year_invalid = ((df["year_arrived_us"] < 1900) | (df["year_arrived_us"] > 2023) | df["year_arrived_us"].isna())
n_year_invalid = year_invalid.sum()
print(f"   Records with year_arrived_us < 1900 or > 2023: {n_year_invalid}")
outlier_exclusions.append(('Year arrived out of range', n_year_invalid))

# Keep only valid years
df_valid_year = df[~year_invalid].copy()
print(f"   Remaining records: {len(df_valid_year)}")

# Combined: valid age AND valid year
print("\n3. COMBINED FILTERING (valid age AND valid year_arrived)")
valid_mask = (~age_invalid) & (~year_invalid)
df = df[valid_mask].copy()
n_excluded_outliers = n_before_outliers - len(df)
print(f"   Records excluded: {n_excluded_outliers}")
print(f"   Records remaining: {len(df)}")

cohort_flow.append({
    'Stage': '2. Outliers removed',
    'N': len(df),
    'Excluded': n_excluded_outliers,
    'Reason': 'Age (0-120) or Year arrived (1900-2023) out of range',
    'Cumulative_N': len(df)
})

## Step 6: Derived Variables & Cross-Validation

In [ ]:
print("\n🔧 DERIVED VARIABLES & VALIDATION")
print("-" * 60)

# Create arrival date
print("\n1. CREATING years_in_us (derived from survey_date and year_arrived_us)")
df["arrival_date"] = pd.to_datetime(
    df["year_arrived_us"].astype("Int64").astype(str) + "-07-01",
    errors="coerce"
)
df["years_in_us"] = (df["survey_date"] - df["arrival_date"]).dt.days / 365.25

# Validation: years_in_us should roughly equal (survey_year - arrival_year)
df["expected_years"] = df["survey_date"].dt.year - df["year_arrived_us"]
df["years_diff"] = (df["years_in_us"] - df["expected_years"]).abs()

# Check for major discrepancies (>1 year difference)
large_discrepancies = (df["years_diff"] > 1).sum()
print(f"   Derived years_in_us from arrival_date")
print(f"   Validation check: Years_in_us vs (survey_year - arrival_year)")
print(f"   ⚠️  Records with >1 year discrepancy: {large_discrepancies}")

if df["years_in_us"].notna().any():
    print(f"   Range: {df['years_in_us'].min():.1f} to {df['years_in_us'].max():.1f} years")
    print(f"   Mean: {df['years_in_us'].mean():.1f} years")

# Apply bounds to years_in_us
print("\n2. BOUNDING years_in_us (expected range: 0-120 years)")
years_invalid = ((df["years_in_us"] < 0) | (df["years_in_us"] > 120))
n_years_invalid = years_invalid.sum()
print(f"   Records with years_in_us < 0 or > 120: {n_years_invalid}")
df.loc[years_invalid, "years_in_us"] = np.nan

# Clean up temporary columns
df = df.drop(columns=["expected_years", "years_diff", "arrival_date"])

print(f"\n✓ Derived variables created and validated")

## Step 7: Cardiovascular Outcome Processing

In [ ]:
print("\n💓 CARDIOVASCULAR OUTCOME PROCESSING")
print("-" * 60)

# Rename CV variables for convenience
cv_rename_map = {
    "have you been diagnosed with any of the following  (choice=heart failure)": "dx_hf",
    "have you been diagnosed with any of the following  (choice=hypertension)": "dx_htn",
    "heart attack (choice=yes)": "hx_mi",
    "stroke (choice=yes)": "hx_stroke",
}

df = df.rename(columns=cv_rename_map)

print(f"\nCV variables renamed for convenience")

# Before conversion: examine raw values
print("\n📊 RAW VALUES BEFORE CONVERSION:")
for col in ["dx_hf", "dx_htn", "hx_mi", "hx_stroke"]:
    if col in df.columns:
        print(f"\n   {col}:")
        print(f"      Non-null count: {df[col].notna().sum()}")
        print(f"      Unique values: {df[col].nunique()}")
        print(f"      Sample values: {df[col].dropna().unique()[:5]}")

# Conversion function
def to_binary(x):
    if pd.isna(x):
        return 0  # NaN/empty = not checked = 0
    s = str(x).strip().lower()
    
    yes_set = {"yes", "y", "true", "1", "1.0", "checked", "check", "x", "selected"}
    no_set = {"no", "n", "false", "0", "0.0", "unchecked", "uncheck", ""}
    
    if s in yes_set:
        return 1
    if s in no_set:
        return 0
    if len(s) > 0:
        return 1  # Any other non-empty value = checked
    
    return 0

# Convert CV outcomes to binary
cv_outcomes = ["dx_hf", "dx_htn", "hx_mi", "hx_stroke"]
print("\n🔄 CONVERTING TO BINARY (0/1):")
for col in cv_outcomes:
    if col in df.columns:
        df[col] = df[col].apply(to_binary)
        n_positive = df[col].sum()
        pct_positive = (df[col].mean() * 100)
        print(f"   {col}: {int(n_positive)} positive ({pct_positive:.1f}%)")

print("\n✓ CV outcomes converted to binary")

## Step 8: Create Year of Arrival Bins

In [ ]:
print("\n📅 CREATING YEAR OF ARRIVAL BINS")
print("-" * 60)

# Distribution of year_arrived_us
print(f"\nDistribution of year_arrived_us (valid records):")
print(df["year_arrived_us"].describe())

# Create 3-bin categorization
df["year_arrived_bin3"] = pd.cut(
    df["year_arrived_us"],
    bins=[0, 2005, 2015, 2025],
    labels=["Before 2005", "2005-2015", "2015-2023"],
    include_lowest=True
)

print(f"\n3-Bin Categorization:")
for bin_name in df["year_arrived_bin3"].cat.categories:
    n_bin = (df["year_arrived_bin3"] == bin_name).sum()
    pct = n_bin / len(df) * 100
    print(f"   {bin_name}: {n_bin} ({pct:.1f}%)")

# Missing
n_missing_bin = df["year_arrived_bin3"].isna().sum()
print(f"   (Missing): {n_missing_bin}")

## Step 9: Create Analysis Sample (Final Filtering)

In [ ]:
print("\n🎯 CREATING FINAL ANALYSIS SAMPLE")
print("-" * 60)

n_before_final = len(df)

# Requirement: valid year_arrived_bin3 (i.e., valid year_arrived_us)
df_analysis = df[df["year_arrived_bin3"].notna()].copy()
n_final = len(df_analysis)
n_excluded_final = n_before_final - n_final

print(f"\nFinal filtering criterion: Valid year_arrived_bin3")
print(f"   Records before: {n_before_final}")
print(f"   Records excluded: {n_excluded_final}")
print(f"   Analysis sample: {n_final}")

cohort_flow.append({
    'Stage': '3. Final analysis sample',
    'N': n_final,
    'Excluded': n_excluded_final,
    'Reason': 'Missing year of arrival bin (missing or out of range data)',
    'Cumulative_N': n_final
})

# Create CV burden measures for analysis sample
print("\n📊 Creating composite CV measures for analysis sample:")
cv_conditions = ["dx_hf", "dx_htn", "hx_mi", "hx_stroke"]
df_analysis["cv_burden_count"] = df_analysis[cv_conditions].sum(axis=1, skipna=True)
df_analysis["any_cv_condition"] = (df_analysis["cv_burden_count"] > 0).astype(int)
df_analysis["major_cv_event"] = ((df_analysis["hx_mi"] == 1) | (df_analysis["hx_stroke"] == 1)).astype(int)

print(f"   CV burden count created (range: {df_analysis['cv_burden_count'].min():.0f}-{df_analysis['cv_burden_count'].max():.0f})")
print(f"   Any CV condition: {df_analysis['any_cv_condition'].sum()} ({df_analysis['any_cv_condition'].mean()*100:.1f}%)")
print(f"   Major CV event: {df_analysis['major_cv_event'].sum()} ({df_analysis['major_cv_event'].mean()*100:.1f}%)")

## Cohort Flow Diagram

In [ ]:
# Create cohort flow table
cohort_flow_df = pd.DataFrame(cohort_flow)

print("\n" + "="*80)
print("COHORT FLOW SUMMARY")
print("="*80)

gt_flow = (
    GT(cohort_flow_df)
    .tab_header(title="Phase 0 Cohort Flow: From Raw Data to Analysis Sample")
    .cols_label(
        Stage="Stage",
        N="Sample Size (N)",
        Excluded="Excluded (N)",
        Reason="Exclusion Reason",
        Cumulative_N="Cumulative N"
    )
    .fmt_integer(columns=["N", "Excluded", "Cumulative_N"])
)

gt_flow

## Cohort Flow Visualization

In [ ]:
# Create visual cohort flow
fig, ax = plt.subplots(figsize=(12, 8))

stages = cohort_flow_df['Stage'].tolist()
sample_sizes = cohort_flow_df['N'].tolist()
excluded = cohort_flow_df['Excluded'].tolist()

y_positions = np.arange(len(stages))[::-1]  # Reverse for top-to-bottom flow

# Plot sample sizes
colors = ['#2ecc71' if e == 0 else '#e74c3c' for e in excluded]
ax.barh(y_positions, sample_sizes, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add labels
for i, (stage, n, ex) in enumerate(zip(stages, sample_sizes, excluded)):
    ax.text(n/2, len(stages)-1-i, f'N={int(n)}', 
            ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    if ex > 0:
        ax.text(n + max(sample_sizes)*0.02, len(stages)-1-i, f'(-{int(ex)})', 
                ha='left', va='center', fontsize=10, color='#e74c3c', fontweight='bold')

ax.set_yticks(y_positions)
ax.set_yticklabels(stages, fontsize=11)
ax.set_xlabel('Sample Size (N)', fontsize=12, fontweight='bold')
ax.set_title('Cohort Flow: Phase 0 Data Cleaning & Selection', fontsize=14, fontweight='bold', pad=20)
ax.set_xlim(0, max(sample_sizes) * 1.15)
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print("✓ Cohort flow visualization complete")

## Missing Data Analysis

In [ ]:
print("\n" + "="*80)
print("MISSING DATA ANALYSIS (Analysis Sample)")
print("="*80)

# Missing data summary
missing_summary = pd.DataFrame({
    'Variable': df_analysis.columns,
    'Missing_N': [df_analysis[col].isna().sum() for col in df_analysis.columns],
    'Missing_%': [df_analysis[col].isna().sum() / len(df_analysis) * 100 for col in df_analysis.columns],
    'Non_Missing_N': [df_analysis[col].notna().sum() for col in df_analysis.columns],
})

missing_summary = missing_summary.sort_values('Missing_%', ascending=False)

gt_missing = (
    GT(missing_summary)
    .tab_header(title="Missing Data Summary (N=" + str(len(df_analysis)) + ")")
    .cols_label(
        Variable="Variable",
        Missing_N="Missing (N)",
        Missing_%="Missing (%)",
        Non_Missing_N="Available (N)"
    )
    .fmt_integer(columns=["Missing_N", "Non_Missing_N"])
    .fmt_number(columns=["Missing_%"], decimals=1)
)

gt_missing

## Data Quality Metrics

In [ ]:
print("\n" + "="*80)
print("DATA QUALITY METRICS")
print("="*80)

print(f"\n✓ SAMPLE RETENTION:")
retention_pct = (n_final / n_initial) * 100
print(f"   Original sample: {n_initial}")
print(f"   Analysis sample: {n_final}")
print(f"   Retention rate: {retention_pct:.1f}%")
print(f"   Overall exclusion rate: {100-retention_pct:.1f}%")

print(f"\n✓ DATA COMPLETENESS (Analysis Sample):")
key_vars = ['survey_date', 'age', 'year_arrived_us', 'years_in_us', 'dx_htn', 'hx_mi', 'hx_stroke']
for var in key_vars:
    if var in df_analysis.columns:
        complete_n = df_analysis[var].notna().sum()
        complete_pct = complete_n / len(df_analysis) * 100
        print(f"   {var}: {complete_pct:.1f}% complete")

print(f"\n✓ OUTCOME PREVALENCE (Analysis Sample):")
print(f"   Hypertension: {df_analysis['dx_htn'].sum()} ({df_analysis['dx_htn'].mean()*100:.1f}%)")
print(f"   Heart attack: {df_analysis['hx_mi'].sum()} ({df_analysis['hx_mi'].mean()*100:.1f}%)")
print(f"   Stroke: {df_analysis['hx_stroke'].sum()} ({df_analysis['hx_stroke'].mean()*100:.1f}%)")
print(f"   Any CV condition: {df_analysis['any_cv_condition'].sum()} ({df_analysis['any_cv_condition'].mean()*100:.1f}%)")

## Data Quality Issues & Recommendations

In [ ]:
print("\n" + "="*80)
print("DATA QUALITY ISSUES & RECOMMENDATIONS")
print("="*80)

issues = []
recommendations = []

# Check for high missing data
for idx, row in missing_summary.iterrows():
    if row['Missing_%'] > 50:
        issues.append(f"⚠️  {row['Variable']}: {row['Missing_%']:.1f}% missing")
        recommendations.append(f"Consider excluding or imputing {row['Variable']}")

# Check for variables with very low variance
if 'dx_hf' in df_analysis.columns:
    hf_prev = df_analysis['dx_hf'].mean()
    if hf_prev < 0.01:
        issues.append(f"⚠️  dx_hf: Only {hf_prev*100:.2f}% prevalence (very rare)")
        recommendations.append("Consider excluding dx_hf due to insufficient variance")

if retention_pct < 80:
    issues.append(f"⚠️  Sample retention: {retention_pct:.1f}% (>20% excluded)")
    recommendations.append("Review exclusion criteria; consider sensitivity analysis with less stringent criteria")

if len(issues) == 0:
    print("\n✓ NO MAJOR DATA QUALITY ISSUES DETECTED")
else:
    print("\n🚨 IDENTIFIED ISSUES:")
    for issue in issues:
        print(f"   {issue}")

if recommendations:
    print("\n💡 RECOMMENDATIONS FOR PHASE 1:")
    for rec in recommendations:
        print(f"   • {rec}")

print("\n✓ Data quality assessment complete")

## Summary Statistics (Analysis Sample)

In [ ]:
print("\n" + "="*80)
print("ANALYSIS SAMPLE SUMMARY STATISTICS")
print("="*80)

# Demographics
print(f"\n👥 DEMOGRAPHICS:")
print(f"   Sample size: {len(df_analysis)}")
print(f"   Age, mean (SD): {df_analysis['age'].mean():.1f} ({df_analysis['age'].std():.1f})")
print(f"   Age range: {df_analysis['age'].min():.0f}-{df_analysis['age'].max():.0f}")

# Immigration variables
print(f"\n🌍 IMMIGRATION VARIABLES:")
print(f"   Years in US, mean (SD): {df_analysis['years_in_us'].mean():.1f} ({df_analysis['years_in_us'].std():.1f})")
print(f"   Year arrived range: {df_analysis['year_arrived_us'].min():.0f}-{df_analysis['year_arrived_us'].max():.0f}")
print(f"   Median year arrived: {df_analysis['year_arrived_us'].median():.0f}")

print(f"\n📊 YEAR OF ARRIVAL DISTRIBUTION:")
for bin_name in df_analysis["year_arrived_bin3"].cat.categories:
    n_bin = (df_analysis["year_arrived_bin3"] == bin_name).sum()
    pct = n_bin / len(df_analysis) * 100
    print(f"   {bin_name}: {n_bin} ({pct:.1f}%)")

# Save analysis sample for Phase 0 report
df_analysis.to_csv('phase0_analysis_sample.csv', index=False)
print(f"\n✓ Analysis sample saved to: phase0_analysis_sample.csv")